In [19]:
from __future__ import print_function
import argparse
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
cudnn.fastest = True
import torchvision.utils as vutils
from torch.autograd import Variable

import models.UNet as net

import models.UNet as net_test
from misc import *
from torchsummary import summary
import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import torch.nn.functional as F
import torch.optim as optim
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from basic_toolfunc import normalization_test, local_area, coord_trans, number_to_coord
from model_test_func import prior_update_gan_noise
from models.gan_models import G, D
from posterior_func import ConvNet_nin, prior_update_gan_noise, ensemble_learning, meas_data_read, obs_list_form, obs_list_form_hxy, two_model_merge, pos_ensemble_gan_noise

In [15]:
index_list = np.load('index_list_pos.npy')
test_cov = np.load('ensemble_learning_cov.npy')
obs_list = np.load('obs_list_pos.npy')
obs_var_list = np.array([0.000001, 0.0001, 0.0001])
point_list_2 = []
for i in range(260):
    x, y = index_list[i,0], index_list[i, 1]
    point_list_2.append([x,y,'h'])
    point_list_2.append([x,y,'x'])
    point_list_2.append([x,y,'y'])

c_list_h = np.load('test_code/c_h.npy')
c_list = [number_to_coord(ele) for ele in c_list_h]

b_list_h = np.load('test_code/b_h.npy')
b_list = [number_to_coord(ele) for ele in b_list_h]

a_list_h = np.load('test_code/a_h.npy')
a_list = [number_to_coord(ele) for ele in a_list_h]

c = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        temp = coord_trans(i, j)
        if temp in c_list_h:
            c[i,j] = 1
        if temp in b_list_h:
            c[i,j] = 2
        if temp in a_list_h:
            c[i,j] = 2

channel = c==1
river = c==2
land = c==0

In [16]:
mu = np.load('mu.npy')
var = np.load('var.npy')
# k = 102
for j in range(20):
    data_inp = []
    for i in range(100*j, 100*j+1):
        temp_data_ = np.load('/home/gtx1080/Sync/Kun/30_min_ele/non_ele_whole/test/{0}_inp.npy'.format(i+10000))
        temp_data_ = temp_data_.transpose([2,0,1])
        data_inp.append(temp_data_)
    data_inp = np.array(data_inp)

    data_op = []
    for i in range(100*j, 100*j+1):
        temp_data_ = np.load('/home/gtx1080/Sync/Kun/30_min_ele/non_ele_whole/test/{0}_op.npy'.format(i+10000))
        temp_data_ = temp_data_.transpose([2,0,1])
        data_op.append(temp_data_)
    data_op = np.array(data_op)
    data_op[:,0,:,:] = data_op[:,0,:,:]*np.sqrt(var[5,0]) + mu[5,0]
    data_op[:,1,:,:] = data_op[:,1,:,:]*np.sqrt(var[6,0]) + mu[6,0]
    data_op[:,2,:,:] = data_op[:,2,:,:]*np.sqrt(var[7,0]) + mu[7,0]
    
    data_inp[:,0,:,:] = data_inp[:,0,:,:]*np.sqrt(var[0,0]) + mu[0,0]
    data_inp[:,1,:,:] = data_inp[:,1,:,:]*np.sqrt(var[1,0]) + mu[1,0]
    data_inp[:,2,:,:] = data_inp[:,2,:,:]*np.sqrt(var[2,0]) + mu[2,0]
    data_inp[:,3,:,:] = data_inp[:,3,:,:]*np.sqrt(var[3,0]) + mu[3,0]
    data_inp[:,4,:,:] = data_inp[:,4,:,:]*np.sqrt(var[4,0]) + mu[4,0]
    
#     op = prior_update_gan_noise(data_inp, k)
    op = ensemble_learning(data_inp[:,0:3,:,:], data_inp[:, 3:5,:,:], 
                           200, 18, index_list, point_list_2, 
                           test_cov, obs_var_list)
    op[0][:,0,:,:] = op[0][:,0,:,:]*np.sqrt(var[5,0]) + mu[5,0]
    op[0][:,1,:,:] = op[0][:,1,:,:]*np.sqrt(var[6,0]) + mu[6,0]
    op[0][:,2,:,:] = op[0][:,2,:,:]*np.sqrt(var[7,0]) + mu[7,0]

780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
780
(260, 1, 3)
(1, 3, 100, 100)
(1, 2, 100, 100)
